# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Импорт и обзор данных

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1uJQ6dv3fKmHN27gxvpyiTdSy1z3m7wFX')

In [3]:
# используем функцию для первичного просмотра данных
def data_review(data):
    display(data.sample(7))
    print('-'*100)
    data.info()
    print('-'*100)
    print('Пропуски в данных')
    display(data.isna().mean().sort_values(ascending=False))
    print('-'*100)
    print(f'Количество явных дубликатов - {data.duplicated().sum()}')
    print('-'*100)
    display(data.describe())

In [4]:
data_review(df)

,calls,minutes,messages,mb_used,is_ultra
2746,56.0,458.22,33.0,19012.75,0
19,49.0,341.67,81.0,11770.28,1
170,27.0,179.47,82.0,19631.63,0
2388,3.0,35.47,62.0,11774.26,1
902,98.0,657.01,70.0,13309.90,0
939,63.0,457.86,47.0,15708.64,0
2216,28.0,166.74,0.0,15887.44,0


----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
----------------------------------------------------------------------------------------------------
Пропуски в данных


calls       0.0
minutes     0.0
messages    0.0
mb_used     0.0
is_ultra    0.0
dtype: float64

----------------------------------------------------------------------------------------------------
Количество явных дубликатов - 0
----------------------------------------------------------------------------------------------------


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Данные загрузились корректно. Типы данных корректны. Пропусков и дубликатов не обнаружено.  
Мы имеем на входе для машинного обучения 3214 объект с 5 признаками, из которых целевой - принадлежность к тарифу.
Причем отметим, что целевой признак среди объектов распределен неравномерно - 70% - тариф "Смарт" и 30% - тариф "Ультра"

## Разделение данных на выборки

Разделим наши данные на признаки и целевой признак.

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

И поделим эти массивы на обучающий, валидационный и тестовый в соотношении 60%:20%:20%

In [6]:
# сначала поделим на обучающий + валидационный и тестовый
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.20, 
                                                                            random_state=10)
# затем поделим на обучающий и валидационный
features_train, features_val, target_train, target_val = train_test_split(features_train, 
                                                                          target_train, 
                                                                          test_size=0.25, 
                                                                          random_state=10)

Проверим корректность разделения

In [7]:
print(f'Обучающая выборка - {(len(features_train) / len(features)):.0%}')
print(f'Валидационная выборка - {(len(features_val) / len(features)):.0%}')
print(f'Тестовая выборка - {(len(features_test) / len(features)):.0%}')

Обучающая выборка - 60%
Валидационная выборка - 20%
Тестовая выборка - 20%


In [8]:
target_train.mean(), target_val.mean(), target_test.mean()

(0.3086099585062241, 0.3110419906687403, 0.2954898911353033)

Соотношение целевого признака по выборкам осталось на уровне общего массива - 30%

## Исследование модели

### Дерево решений

Сначала исследуем модель дерева решений и подберем для нее гиперпараметры.

In [9]:
best_dtc_model = None
best_dtc_accuracy = 0
best_dtc_depth = 0
best_dtc_mss = 0
best_dtc_msl =0
for mss in tqdm(range(2,10)):
    for msl in range(1,20):
        for depth in range(1,10):   
            model = DecisionTreeClassifier(random_state=10, max_depth=depth, min_samples_leaf=msl, min_samples_split=mss)
            model.fit(features_train, target_train)
            predictions_val = model.predict(features_val)
            if accuracy_score(target_val, predictions_val) > best_dtc_accuracy:
                best_dtc_accuracy = accuracy_score(target_val, predictions_val)
                best_dtc_model = model
                best_dtc_depth = depth
                best_dtc_mss = mss
                best_dtc_msl = msl
print(f'Максимальная доля правильных ответов - {best_dtc_accuracy:.0%}')
print(f'Оптимальная максимальная глубина дерева решений - {best_dtc_depth}')
print(f'Оптимальное минимальное число объектов в узле дерева - {best_dtc_mss}')
print(f'Оптимальное минимальное число объектов в листьях дерева - {best_dtc_msl}')    

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:05<00:00,  1.43it/s]

Максимальная доля правильных ответов - 80%
Оптимальная максимальная глубина дерева решений - 7
Оптимальное минимальное число объектов в узле дерева - 2
Оптимальное минимальное число объектов в листьях дерева - 18


### Случайный лес

Теперь рассмотрим модель случайного леса и подберем для нее гиперпараметры (ограничимся только количеством деревьев и глубиной дерева решений)

In [10]:
best_rfc_model = None
best_rfc_accuracy = 0
best_rfc_n_est = 0 
best_rfc_depth = 0
best_rfc_mss = 0
best_rfc_msl = 0
for n_est in tqdm(range(10,201,10)):
    for depth in range(1,10):   
        model = RandomForestClassifier(random_state=10, max_depth=depth, n_estimators=n_est)
        model.fit(features_train, target_train)
        predictions_val = model.predict(features_val)
        if accuracy_score(target_val, predictions_val) > best_dtc_accuracy:
            best_rfc_accuracy = accuracy_score(target_val, predictions_val)
            best_rfc_model = model
            best_rfc_n_est = n_est
            best_rfc_depth = depth
            
print(f'Максимальная доля правильных ответов - {best_rfc_accuracy:.0%}')
print(f'Оптимальное количество деревьев в лесу - {best_rfc_n_est}')
print(f'Оптимальная максимальная глубина дерева решений - {best_rfc_depth}')
  

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:34<00:00,  1.72s/it]

Максимальная доля правильных ответов - 80%
Оптимальное количество деревьев в лесу - 170
Оптимальная максимальная глубина дерева решений - 8


### Логистическая регрессия

Еще проверим как отработает модель логистической регрессии

In [11]:
model = LogisticRegression(random_state=10, intercept_scaling= 10)
model.fit(features_train, target_train)
predictions_val = model.predict(features_val)
best_lr_accuracy = accuracy_score(target_val, predictions_val)
print(f'Доля правильных ответов - {best_lr_accuracy:.0%}')

Доля правильных ответов - 70%


### Выбор лучшей модели.

Выберем лучшую из моделей по наибольшей доле правильных ответов.

In [12]:
if best_dtc_accuracy > best_rfc_accuracy:
    best_model = best_dtc_model
    print(f"Лучшая модель - 'дерево решений' c гиперпараметрами:")
    print(f'Оптимальная максимальная глубина дерева решений - {best_dtc_depth}')
    print(f'Оптимальное минимальное число объектов в узле дерева - {best_dtc_mss}')
    print(f'Оптимальное минимальное число объектов в листьях дерева - {best_dtc_msl}') 
else:
    if best_rfc_accuracy > best_lr_accuracy:
        best_model = best_rfc_model
        print(f"Лучшая модель - 'случайный лес' c гиперпараметрами:")
        print(f'Оптимальное количество деревьев в лесу - {best_rfc_n_est}')
        print(f'Оптимальная максимальная глубина дерева решений - {best_rfc_depth}')
    else:
        best_model = best_lr_model
        print(f"Лучшая модель - 'логистическая регрессия'")    

Лучшая модель - 'случайный лес' c гиперпараметрами:
Оптимальное количество деревьев в лесу - 170
Оптимальная максимальная глубина дерева решений - 8


В итоге модели дерева решений и случайного леса на порядок оказались лучше модели логистической регрессии (по 80% против 70% правильных ответов соответственно). Правильность у этих двух моделей практически идентична, разница в десятых долях процента, но все же в пользу модели случайного леса. Останавливаем свой выбор именно на ней с вышеуказанными гиперпараметрами, но отмечаем, обучение ей занимает больше времени.

## Проверка модели на тестовой выборке

Проверим нашу лучшую модель на тестовой выборке.

In [13]:
predictions_test = best_model.predict(features_test)
print(f'Доля правильных ответов в тестовой выборке - {accuracy_score(target_test, predictions_test):.0%}')

Доля правильных ответов в тестовой выборке - 82%


Доля правильных ответов получилась даже чуть выше, чем на валидационной выборке.

## Проверка модели на адекватность

Проверим нашу модель на адекватность путем сравнения данных полученных по ней со случайно распределенными данными.

In [14]:
# сгенерируем случайно распределенный массив "0" и "1" с вероятностью "1" равной доле "1" в нашей тестовой выборке
random_test = np.random.binomial(1, p=target_test.mean(), size=len(target_test))

In [15]:
print(f'Доля правильных ответов в случайной выборке - {accuracy_score(target_test, random_test):.0%}')

Доля правильных ответов в случайной выборке - 57%


Доля правильных ответов в случайной выборке мала. Чтобы исключить случайность величны этого процента, повторим эксперимент со случайной выборкой большее количество раз (10 тысяч к примеру) и выберем наилучший результат.

In [16]:
best_rnd_accuracy = 0
for _ in range (1,10000):
    random_test = np.random.binomial(1,p=target_test.mean(), size=len(target_test))
    if accuracy_score(target_test, random_test) > best_rnd_accuracy:
        best_rnd_accuracy = accuracy_score(target_test, random_test)
print(f'Максимальная доля правильных ответов в случайной выборке - {best_rnd_accuracy:.0%}')    

Максимальная доля правильных ответов в случайной выборке - 65%


Попробуем еще сравнить нашу модель с моделью, которая значение целевого признака берет от моды по выборке (в нашем случае 0) 

In [17]:
zero_test = np.array([0]*len(target_test))
print(f'Доля правильных ответов в выборке по моде - {accuracy_score(target_test, zero_test):.0%}')

Доля правильных ответов в выборке по моде - 70%


Максимально достижимая доля правильных ответов в случайно сгенерированной выборке (даже с учетом распределения целевого признака) составляет 65%. В выборке по моде - 70%. А у нас в лучшей моделе доля правильных ответов - 82%, что говорит нам о праве нашей моделе на существование и то, что она вполне адекватно подбирает тариф для пользователя исходя из предоставленных ей для анализа признаков.